In [ ]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
from matplotlib import pyplot as plt
import seaborn as sns
import pylab as py
import time
import cv2
import scipy.io as spio

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# set your device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#Architecture 1
class Generator(nn.Module):
    def __init__(self, in_dim, out_dim, hid_dim=40, num_layers=5):
        super(Generator, self).__init__()
        
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        
        self.layers = nn.ModuleList()
        block_layer = block(in_dim, hid_dim)
        for layer in block_layer:
            self.layers.append(layer)
            
        for layer_i in range(num_layers - 1):
            block_layer = block(hid_dim, hid_dim)
            for layer in block_layer:
                self.layers.append(layer)
                
        self.layers.append(nn.Linear(hid_dim, out_dim))
        
    def forward(self, data, noise):
        # Concatenate input data and noise to produce output
        gen_input = torch.cat((data, noise), -1)
        
        out = gen_input
        for i in range(len(self.layers)):
            out = self.layers[i](out)
        return out

class Discriminator(nn.Module):
    def __init__(self, in_dim, out_dim = 1, hid_dim=20, num_layers=2):
        super(Discriminator, self).__init__()
        
        self.layers = [nn.Linear(in_dim, hid_dim)]
        self.layers.append(nn.Dropout(0.4))
        self.layers.append(nn.LeakyReLU(0.2, inplace=True))
        
        self.out_dim = out_dim
        self.out_dim = 1
        
        for layer_i in range(num_layers - 1):
            self.layers.append(nn.Linear(hid_dim, hid_dim))
            self.layers.append(nn.Dropout(0.4))
            self.layers.append(nn.LeakyReLU(0.2, inplace=True))
            
        self.layers.append(nn.Linear(hid_dim, self.out_dim))
        self.model = nn.Sequential(*self.layers)
        
    def forward(self, data, out, phy):
        # Concatenate data and out to produce input
        d_in = torch.cat((data, out, phy), -1)
        d_out = self.model(d_in)
        return d_out
    
    
# x,y -> z
class Q_Net(nn.Module):
    def __init__(self):
        super(Q_Net, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        
        self.model = nn.Sequential(
            *block(data_dim + out_dim, 20, normalize=False),
            *block(20, 20),
            nn.Linear(20, noise_dim)
        )

    def forward(self, data, output):
        # Concatenate input data and noise to produce output
        gen_input = torch.cat((data, output), -1)
        noise = self.model(gen_input)
        return noise

In [ ]:
### Real = 0, fake = 1
def discriminator_loss(logits_real, logits_fake, logits_fake_unlabelled):
    
    loss = - torch.mean(torch.log(1.0 - torch.sigmoid(logits_real) + 1e-8) + torch.log(torch.sigmoid(logits_fake) + 1e-8)) \
            - torch.mean(torch.log(torch.sigmoid(logits_fake_unlabelled) + 1e-8))
    return loss

def generator_loss(logits_fake, logits_fake_unlabelled):
    
    loss = torch.mean(logits_fake) + torch.mean(logits_fake_unlabelled)
    return loss

In [ ]:
def physics_loss(x, y, stat_x = [0,1], stat_y = [0,1]):  #stat [0] = mean, stat [1] = std
    stat_x = torch.Tensor(stat_x).to(device)
    stat_y = torch.Tensor(stat_y).to(device)
    x = x * stat_x[1] + stat_x[0]
    y = y * stat_y[1] + stat_y[0]
    energy_loss =   - 0.5*x[:,4:5]*torch.pow(x[:,0:1],2) \
                    - 0.5*x[:,5:6]*torch.pow(x[:,1:2],2) \
                    + 0.5*x[:,4:5]*torch.pow(y[:,0:1],2) \
                    + 0.5*x[:,5:6]*torch.pow(y[:,1:2],2)
                  
    
    momentum_loss =  - x[:,4:5] * x[:,0:1] \
                     - x[:,5:6] * x[:,1:2] \
                     + x[:,4:5] * y[:,0:1] \
                     + x[:,5:6] * y[:,1:2] 
    f = torch.cat([energy_loss, momentum_loss], dim = 1)
    return f

def expo_transformation(lambda_phy, phyloss):
    probs = torch.exp(-lambda_phy * torch.abs(phyloss))
    return probs

In [ ]:
def sample_noise(batch_size, dim, mean=0, std=1):
    to_return = mean + std * torch.randn((batch_size, dim))
    return to_return

In [ ]:
tr_frac = 0.8
#load data
data = np.loadtxt( '../../datasets/collision_shuffled.txt' )
labels = data[:,-2:]
x = data[:,:-2]


#training and test splits
n_obs = int(tr_frac * x.shape[0])
train_x , train_y = x[:n_obs,:] , labels[:n_obs,:] 
test_x , test_y = x[n_obs:,:] , labels[n_obs:,:] 

#defining noise dimensions
noise_dim = 2
data_dim = train_x.shape[-1]
out_dim = labels.shape[-1]

# Normalization 

# train_x:
mean_x = train_x.mean(axis=0)
std_x = train_x.std(axis=0)

train_x = (train_x-mean_x)/std_x
test_x = (test_x-mean_x)/std_x

# train_y:
mean_y = train_y.mean(axis=0)
std_y = train_y.std(axis=0)

train_y = (train_y-mean_y)/std_y
test_y = (test_y-mean_y)/std_y

#defining batch size and parameters
batch_size = 64 # mini-batch size
num_workers = 4 # how many parallel workers are we gonna use for reading data
shuffle = True # shuffle the dataset

#training and testing dataset creation
train_x = torch.FloatTensor(train_x).to(device)
test_x = torch.FloatTensor(test_x).to(device)
train_y = torch.FloatTensor(train_y).to(device)
test_y = torch.FloatTensor(test_y).to(device)

train_loader = DataLoader(list(zip(train_x,train_y)), batch_size=batch_size, shuffle=shuffle)

#DEFINING MODELS : GENERATOR, DISCRIMINATOR AND Q-NET

D = Discriminator(in_dim = (data_dim + out_dim + 2), out_dim = 1, hid_dim=20, num_layers=2).to(device)
G = Generator(in_dim = (noise_dim + data_dim), out_dim = out_dim, hid_dim=40, num_layers=5).to(device)
Q = Q_Net().to(device)

D_optimizer = torch.optim.Adam(D.parameters(), lr=1e-3, betas = (0.5, 0.999))
G_optimizer = torch.optim.Adam(G.parameters(), lr=1e-3, betas = (0.5, 0.999))
Q_optimizer = torch.optim.Adam(Q.parameters(), lr=1e-3, betas = (0.5, 0.999))

###############################################################################################
######################################  TRAINING PARAMETERS ###################################

num_epochs = 5000
lambda_mse = 0
lambda_phy = 0
lambda_prob = 0.001
lambda_q = 0.5

###############################################################################################

Adv_loss = np.zeros(num_epochs)
G_loss = np.zeros(num_epochs)
D_loss = np.zeros(num_epochs)
Q_loss = np.zeros(num_epochs)
MSE_loss = np.zeros(num_epochs)
PHY_loss = np.zeros(num_epochs)

G_loss_batch = []
D_loss_batch = []

train_pred = np.zeros((num_epochs,train_y.shape[0],out_dim))
test_pred = np.zeros((num_epochs,test_y.shape[0],out_dim))

for epoch in range(num_epochs):
    epoch_loss = 0
    ## make directory to store results
    for i, (x, y) in enumerate(train_loader):

        # DISCRIMINATOR UPDATE
        D_optimizer.zero_grad()

        ## REAL DATA 
        real_prob = expo_transformation(lambda_prob, physics_loss(x,y, [mean_x, std_x], [mean_y, std_y]))
        real_logits = D.forward(x, y, real_prob)

        ## PREDICTED SAMPLES FROM BATCH
        D_noise = sample_noise(x.shape[0], noise_dim).to(device)
        y_pred = G.forward(x, D_noise)
        phy_prob = expo_transformation(lambda_prob, physics_loss(x,y_pred, [mean_x, std_x], [mean_y, std_y]))
        fake_logits = D.forward(x, y_pred, phy_prob)

        ## UNLABELLED DATA SAMPLES
        D_noise_unlabelled = sample_noise(test_x.shape[0], noise_dim).to(device)
        y_pred_unlabelled = G.forward(test_x, D_noise_unlabelled)
        phy_prob_unlabelled = expo_transformation(lambda_prob, physics_loss(test_x,y_pred_unlabelled, [mean_x, std_x], [mean_y, std_y]))
        fake_logits_unlabelled = D.forward(test_x, y_pred_unlabelled, phy_prob_unlabelled) 

        d_loss = discriminator_loss(real_logits, fake_logits, fake_logits_unlabelled)


        d_loss.backward()
        D_optimizer.step()

        # GENERATOR UPDATE

        G_optimizer.zero_grad()

        G_noise = sample_noise(x.shape[0], noise_dim).to(device)
        y_pred = G.forward(x, G_noise)
        phy_prob = expo_transformation(lambda_prob, physics_loss(x,y_pred, [mean_x, std_x], [mean_y, std_y]))
        fake_logits = D.forward(x, y_pred, phy_prob)

        ## UNLABELLED DATA SAMPLES
        G_noise_unlabelled = sample_noise(test_x.shape[0], noise_dim).to(device)
        y_pred_unlabelled = G.forward(test_x, G_noise_unlabelled)
        phy_prob_unlabelled = expo_transformation(lambda_prob, physics_loss(test_x,y_pred_unlabelled, [mean_x, std_x], [mean_y, std_y]))

        n_phy_tot = torch.cat([phy_prob_unlabelled, phy_prob], dim = 0)

        fake_logits_unlabelled = D.forward(test_x, y_pred_unlabelled, phy_prob_unlabelled) 

        mse_loss = torch.nn.functional.mse_loss(y_pred, y)
        phy_loss = torch.mean(torch.abs(physics_loss(x,y_pred, [mean_x, std_x], [mean_y, std_y])))  ### can also compute physics loss on the entire set.

        z_pred = Q.forward(x, y_pred)
        mse_loss_Z = torch.nn.functional.mse_loss(z_pred, G_noise)

        adv_loss = generator_loss(fake_logits, fake_logits_unlabelled) 
        g_loss = adv_loss + lambda_mse * mse_loss + lambda_phy * phy_loss + lambda_q * mse_loss_Z

        g_loss.backward()
        G_optimizer.step()

        Q_optimizer.zero_grad()
        Q_noise = sample_noise(x.shape[0], noise_dim).to(device)
        y_pred = G.forward(x, Q_noise)
        z_pred = Q.forward(x, y_pred)
        q_loss = torch.nn.functional.mse_loss(z_pred, Q_noise)
        q_loss.backward()
        Q_optimizer.step()

        G_loss_batch.append(g_loss.detach().cpu().numpy())
        D_loss_batch.append(d_loss.detach().cpu().numpy())

        Adv_loss[epoch] += adv_loss.detach().cpu().numpy()
        MSE_loss[epoch] += mse_loss.detach().cpu().numpy()
        G_loss[epoch] += g_loss.detach().cpu().numpy()
        D_loss[epoch] += d_loss.detach().cpu().numpy()
        Q_loss[epoch] += q_loss.detach().cpu().numpy()
        PHY_loss[epoch] += phy_loss.detach().cpu().numpy()

    if (epoch % 100 == 0):
        print(
            "[Epoch %d/%d] [MSE loss: %f] [G loss: %f] [D loss: %f] [Q loss: %f] [Phy loss: %f] [Adv G loss: %f]"
            % (epoch, num_epochs, MSE_loss[epoch], G_loss[epoch], D_loss[epoch], Q_loss[epoch], PHY_loss[epoch], Adv_loss[epoch] )
        )

    G_train_noise = sample_noise(train_x.shape[0], noise_dim).to(device)
    train_pred[epoch,:,:] = G.forward(train_x, G_train_noise).detach().cpu().numpy()

    G_test_noise = sample_noise(test_x.shape[0], noise_dim).to(device)
    test_pred[epoch,:,:] = G.forward(test_x, G_test_noise).detach().cpu().numpy()

# ###############################################################################################
# ######################################## LOSS PLOTS ###########################################
plt.figure(figsize=(10,10))
plt.plot(Adv_loss)
plt.plot(MSE_loss)
plt.plot(G_loss)
plt.plot(D_loss)
plt.plot(PHY_loss)
plt.legend(['Adv_loss','MSE_loss','G_loss','D_loss','PHY_loss'])
#     plt.savefig("Vizualization/"+str(experiment_name)+"Loss_plot.jpg")
plt.show()
# ###############################################################################################

# ###############################################################################################
# ################################## TEST PREDICTIONS ###########################################

n_samples = 10000
test_samples = np.zeros((n_samples, test_y.shape[0], test_y.shape[1]))
print(test_samples.shape)

for i in range(n_samples):
    G_test_noise = sample_noise(test_x.shape[0], noise_dim).to(device)
    test_samples[i,:,:] = G.forward(test_x, G_test_noise).detach().cpu().numpy()*std_y+mean_y


test_mean_y_0 = np.mean(test_samples, 0)[:, 0].flatten()
test_mean_y_1 = np.mean(test_samples, 0)[:, 1].flatten()

test_std_y_0 = np.std(test_samples, 0)[:, 0].flatten()
test_std_y_1 = np.std(test_samples, 0)[:, 1].flatten()

# print(test_y.shape)
test_y_true_0 = test_y[:, 0].detach().cpu().numpy()*std_y[0] + mean_y[0]
test_y_true_1 = test_y[:, 1].detach().cpu().numpy()*std_y[1] + mean_y[1]

x = np.linspace(0, test_y.shape[0]-1, test_y.shape[0])


plt.figure(figsize=(20,7))
plt.plot(x, test_mean_y_0 , label = 'test predictions', alpha= 0.9, color='b', marker='+')
plt.fill_between(x, test_mean_y_0-2*test_std_y_0, test_mean_y_0+2*test_std_y_0, alpha=0.2, color='b')
# plt.errorbar(x,test_mean_y_0,test_std_y_0)
plt.plot(x, test_y_true_0, label = 'ground truth', alpha=1, color='r', marker='*')
py.legend(loc='upper right')
#     plt.savefig("Vizualization/"+str(experiment_name)+"Test_predictions_1.jpg")
plt.show()

plt.figure(figsize=(20,7))
plt.plot(x, test_mean_y_1 , label = 'test predictions', alpha= 0.9, color='b', marker='+')
plt.fill_between(x, test_mean_y_1-2*test_std_y_1, test_mean_y_1+2*test_std_y_1, alpha=0.2, color='b')
# plt.errorbar(x,mean_y,std_y)
plt.plot(x, test_y_true_1, label = 'ground truth', alpha=1, color='r', marker='*')
py.legend(loc='upper right')
#     plt.savefig("Vizualization/"+str(experiment_name)+"Test_predictions_2.jpg")
plt.show()

test_x = test_x.detach().cpu().numpy()
test_x = test_x * std_x + mean_x

test_rmse0 = (((test_mean_y_0.flatten() - test_y_true_0.flatten())**2).mean())**0.5
test_rmse1 = (((test_mean_y_1.flatten() - test_y_true_1.flatten())**2).mean())**0.5
# test_rmse = (((np.stack(test_mean_y_0,test_mean_y_1) - test_y_true_1.flatten())**2).mean())**0.5

energy_loss = np.mean(np.absolute(0.5*test_x[:,4]*np.power(test_x[:,0],2)+0.5*test_x[:,5]*np.power(test_x[:,1],2)
                                  -0.5*test_x[:,4]*np.power(test_mean_y_0,2)-0.5*test_x[:,5]*np.power(test_mean_y_1,2)))

momentum_loss = np.mean(np.absolute(test_x[:,4]*test_x[:,0] + test_x[:,5]*test_x[:,1] 
                                     - test_x[:,4]*test_mean_y_0 - test_x[:,5]*test_mean_y_1))
test_phy_loss = (energy_loss + momentum_loss)

test_true = np.stack((test_y_true_0, test_y_true_1), axis =-1).flatten()
test_mean_y = np.stack((test_mean_y_0, test_mean_y_1), axis =-1).flatten()
test_rmse = ((( test_mean_y - test_true)**2).mean())**0.5

print("test RMSE = %f" %(test_rmse))
print("test RMSE va = %f" %(test_rmse0))
print("test RMSE vb = %f" %(test_rmse1))
print("test Physical Inconsistency = %f" %(test_phy_loss))


energy_loss_true = np.mean(np.absolute(0.5*test_x[:,4]*np.power(test_x[:,0],2)+0.5*test_x[:,5]*np.power(test_x[:,1],2)-0.5*test_x[:,4]*np.power(test_y_true_0,2)-0.5*test_x[:,5]*np.power(test_y_true_1,2)))

momentum_loss_true = np.mean(np.absolute(test_x[:,4]*test_x[:,0] + test_x[:,5]*test_x[:,1] 
                                     - test_x[:,4]*test_y_true_0 - test_x[:,5]*test_y_true_1))
test_phy_loss_true = (energy_loss_true + momentum_loss_true)
print("test Physical Inconsistency (TRUE)= %f" %(test_phy_loss_true))


